# Lesson 2: Memory
<a target="_blank" href="https://colab.research.google.com/github/bqtankiet/langchain-llm-course/blob/main/L2_Memory.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook documents my learning journey on **LangChain for LLM Application Development** course from Deeplearning.ai \
[Lesson 2: Memory](https://learn.deeplearning.ai/courses/langchain/lesson/ls57z/memory)

\
What I Learned
- The Importance of Memory: I learned why memory is crucial for building conversational AI and how LLMs are inherently stateless.
- Manual Memory Implementation: I saw how to build a basic memory system from scratch using a simple Python list to store conversation history.
- Legacy LangChain Memory (Deprecated): I explored the now-deprecated memory modules in LangChain, such as `ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationTokenBufferMemory`, and `ConversationSummaryBufferMemory`, to understand the evolution of memory management.
- The Modern Approach with `RunnableWithMessageHistory`: I learned how to use the recommended `RunnableWithMessageHistory` to create stateful, session-aware conversational chains.
- Customizing Memory with `BaseChatMessageHistory`: I discovered how to create a custom memory class by inheriting from `BaseChatMessageHistory`, allowing for tailored memory management and summarization logic.


## I. Setting up the Environment

In [ ]:
!pip install -qU python-dotenv
!pip install -qU langchain-groq

In [ ]:
import os

from dotenv import load_dotenv
_ = load_dotenv(override=True) # read local .env file

## II. Define our ChatModel

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model = "llama-3.3-70b-versatile",
    model_provider = "groq",
    temperature = 0
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a simple assistant. Respond in a simple and very short way."),
    ("human", "{input}")
])

chain = prompt | llm

## III. What happens if we don't use Memory?

In [ ]:
from langchain_core.messages import HumanMessage
def simple_chat(text):
  response = chain.invoke([HumanMessage(content=text)])
  print(response.content)

In [ ]:
simple_chat("Hello, my name is Ken")

Hi Ken.


In [ ]:
simple_chat("What is 1+1?")

2


In [ ]:
simple_chat("What is my name?") # The LLM don't remember my name

I don't know.


## IV. Let's try to build a Simple Memory


In [ ]:
messages = []

def memory_chat(text):
    global messages
    messages.append(HumanMessage(content=text))
    response = chain.invoke(messages)
    messages.append(response)
    print(response.content)

In [ ]:
memory_chat("Hello, my name is Ken")

Hi Ken.


In [ ]:
memory_chat("What is 1+1?")

2


In [ ]:
memory_chat("What is my name?") # Yeah! The model remembers my name correctly

Ken


In [ ]:
# Just see the overall conversation memory
for m in messages:
  role = m.__class__.__name__
  print(f'{role}: {m.content}')

HumanMessage: Hello, my name is Ken
AIMessage: Hi Ken.
HumanMessage: What is 1+1?
AIMessage: 2
HumanMessage: What is my name?
AIMessage: Ken


## V. Experimenting with deprecated LangChain Memory Types
Note: All ConversationMemory types used below are deprecated in LangChain v0.3+
It's recommended to use LCEL with `RunnableWithMessageHistory` and manage trimming/summarizing yourself.


### ConversationBufferMemory
(deprecated)

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chain,
    memory = memory,
    verbose=True
)

/tmp/ipython-input-16-1620522523.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-16-1620522523.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [ ]:
conversation.predict(input="Hi, my name is Ken")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Ken
AI:

> Finished chain.


'Hello Ken, nice to meet you.'

In [ ]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ken
AI: Hello Ken, nice to meet you.
Human: What is 1+1?
AI:

> Finished chain.


'The answer is 2.'

In [ ]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Ken
AI: Hello Ken, nice to meet you.
Human: What is 1+1?
AI: The answer is 2.
Human: What is my name?
AI:

> Finished chain.


'Ken.'

In [ ]:
print(memory.buffer)

Human: Hi, my name is Ken
AI: Hello Ken, nice to meet you.
Human: What is 1+1?
AI: The answer is 2.
Human: What is my name?
AI: Ken.


In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is Ken\nAI: Hello Ken, nice to meet you.\nHuman: What is 1+1?\nAI: The answer is 2.\nHuman: What is my name?\nAI: Ken.'}

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

Human: Hi
AI: What's up


In [ ]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [ ]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [ ]:
conversation = ConversationChain(
    llm=chain,
    memory = memory,
    verbose=True
)
conversation.predict(input="What did you say when I said Hi")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What did you say when I said Hi
AI:

> Finished chain.


'I said "What\'s up".'

### ConversationBufferWindowMemory
(deprecated)

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

/tmp/ipython-input-30-802562642.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [ ]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=chain,
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Hi, my name is Ken")

'Hello Ken, nice to meet you.'

In [ ]:
conversation.predict(input="What is 1+1?")

'The answer is 2.'

In [ ]:
conversation.predict(input="What is my name?")

"I don't know."

### ConversationTokenBufferMemory
(deprecated)

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=25)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})

/tmp/ipython-input-38-2862484728.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=25)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### ConversationSummaryMemory
(deprecated)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

/tmp/ipython-input-41-1387621936.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [ ]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI, and they exchange casual small talk, with the human eventually asking about the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI, and they exchange casual small talk, with the human eventually asking about the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the demo, I would recommend showing the latest advancements in natural language processing, specifically the capabilities of our LLM (Large Language Model) in generating human-like text and answering complex questions. You could also showcase its ability to understand and respond to voice commands, which is a key feature that has been improved in the latest update.\n\nOne idea for a demo could be to ask the customer to provide a topic or question, and then use the LLM to generate a short article or response on the spot. This would demonstrate the model\'s ability to think creatively and provide relevant information in real-time.\n\nAdditionally, you could also show some of the pre-built applications that we have developed using the LLM, such as the text summarization tool or the chatbot interface. These demos would give the customer a better understanding of how our technology can be applied in real-world scenarios and provide value to their business.\n\nWe also have some pre-prep

In [ ]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI, and they exchange casual small talk, with the human eventually asking about the schedule for the day. The AI informs the human of a meeting at 8am with the product team, where the human will need to have their PowerPoint presentation prepared, followed by time to work on the LangChain project from 9am-12pm. At noon, the human has lunch with a customer at an Italian restaurant, where they will discuss the latest in AI and demonstrate the latest LLM demo, for which the AI recommends showing the advancements in natural language processing, such as generating human-like text, answering complex questions, and understanding voice commands. The AI suggests demo ideas, including generating a short article or response on the spot, showcasing pre-built applications like text summarization or chatbot interfaces, or using pre-prepared demos like "Conversational AI" or "Language Translation", and asks the human to choose a demo or share their own idea.'

## VI. RunnableWithMessageHistory
`RunnableWithMessageHistory` is the modern approach in LangChain 0.3.x

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser

role = "You are a simple assistant. Respond in a simple and very short way"

prompt = ChatPromptTemplate.from_messages([
    ("system", role),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

parser = StrOutputParser()

chain = prompt | llm | parser

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id):
  if session_id not in chat_map:
    chat_map[session_id] = InMemoryChatMessageHistory()
  return chat_map[session_id]

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

history_chat = RunnableWithMessageHistory(
    chain,
    get_session_history=get_chat_history,
    history_messages_key="history",
    input_message_key="input"
)

In [ ]:
history_chat.invoke(
    {"input": "Hi, my name is Ken"},
    config={"session_id": "111"}
)

'Hi Ken'

In [ ]:
history_chat.invoke(
    {"input": "What is 1+1?"},
    config={"session_id": "111"}
)

'2'

In [ ]:
history_chat.invoke(
    {"input": "What is my name?"},
    config={"session_id": "111"}
)

'Ken'

In [ ]:
history_chat.invoke(
    {"input": "What is my name?"},
    config={"session_id": "222"} # another session_id
)

"I don't know."

In [ ]:
history = chat_map["111"]
print(type(history))
history.messages

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>


[HumanMessage(content='Hi, my name is Ken', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Ken', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Ken', additional_kwargs={}, response_metadata={})]

## Extra: Customizing BaseChatMessageHistory


In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class ConversationSummaryHistory(BaseChatMessageHistory):
  def __init__(self, llm):
    self.messages = []
    self.llm = llm

  def add_message(self, messages):

    # Print the details
    print(self.messages)
    print(f"{messages.type}: {messages.content}")

    summary_prompt = ChatPromptTemplate.from_messages([
        ("system",
          "You are an expert conversation summarizer. Your task is to generate an updated summary "
          "of the conversation, given:\n"
          "- The existing summary\n"
          "- A list of new messages with speaker roles (either 'Human' or 'AI')\n\n"
          "Update the summary to include the new information, keeping it concise but detailed. "
          "Preserve important facts, names, decisions, and questions. "
          "Do not lose prior context. Just return the updated summary without any additional explanation."),
        ("human",
         "Existing conversation summary: \n{summary}\n"
         "{role} messages: \n{messages}")
    ])

    summary_messages = summary_prompt.format_messages(
        summary=self.messages,
        role=messages.type,
        messages=messages.content)
    summary_response = llm.invoke(summary_messages)
    self.messages = [SystemMessage(content = summary_response.content)]

  def clear(self):
    self.messages=[]

In [ ]:
chat_history = {}
def get_chat_history(session_id):
  if session_id not in chat_history:
    chat_history[session_id] = ConversationSummaryHistory(llm)
  return chat_history[session_id]

In [ ]:
summary_history_chat = RunnableWithMessageHistory(
    chain,
    get_session_history=get_chat_history,
    history_messages_key="history",
    input_messages_key="input"
)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

get_chat_history("111").add_message(HumanMessage(content="Hello"))
get_chat_history("111").add_message(AIMessage(content="What's up"))
get_chat_history("111").add_message(HumanMessage(content="Not much, just hanging"))
get_chat_history("111").add_message(AIMessage(content="Cool"))
get_chat_history("111").add_message(HumanMessage(content="What is on the schedule today?"))
get_chat_history("111").add_message(AIMessage(content=schedule))

[]
human: Hello
[SystemMessage(content='The conversation started with a greeting from a human who said "Hello".', additional_kwargs={}, response_metadata={})]
ai: What's up
[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up".', additional_kwargs={}, response_metadata={})]
human: Not much, just hanging
[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging".', additional_kwargs={}, response_metadata={})]
ai: Cool
[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging". The AI then said "Cool".', additional_kwargs={}, response_metadata={})]
human: What is on the schedule today?
[SystemMessage(content='The conversation started with a greeting from a human who said "

In [ ]:
get_chat_history("111").messages

[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging". The AI then said "Cool". The human asked about their schedule, inquiring "What is on the schedule today?" The AI replied that there is a meeting at 8am with the product team, requiring a prepared PowerPoint presentation. From 9am-12pm, the human has time to work on their LangChain project. At Noon, the human has lunch at the Italian restaurant with a customer, who is traveling from over an hour away to discuss the latest in AI, and the human should bring their laptop to show the latest LLM demo.', additional_kwargs={}, response_metadata={})]

In [ ]:
summary_history_chat.invoke(
    {"input": "What would be a good demo to show? And why?"},
    config={"session_id":"111"}
)

[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging". The AI then said "Cool". The human asked about their schedule, inquiring "What is on the schedule today?" The AI replied that there is a meeting at 8am with the product team, requiring a prepared PowerPoint presentation. From 9am-12pm, the human has time to work on their LangChain project. At Noon, the human has lunch at the Italian restaurant with a customer, who is traveling from over an hour away to discuss the latest in AI, and the human should bring their laptop to show the latest LLM demo.', additional_kwargs={}, response_metadata={})]
human: What would be a good demo to show? And why?
[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging". The AI then said "Cool". The human as

"LLM text generation demo. It's impressive and relevant."

In [ ]:
get_chat_history("111").messages

[SystemMessage(content='The conversation started with a greeting from a human who said "Hello". The AI responded with "What\'s up". The human then replied, "Not much, just hanging". The AI then said "Cool". The human asked about their schedule, inquiring "What is on the schedule today?" The AI replied that there is a meeting at 8am with the product team, requiring a prepared PowerPoint presentation. From 9am-12pm, the human has time to work on their LangChain project. At Noon, the human has lunch at the Italian restaurant with a customer, who is traveling from over an hour away to discuss the latest in AI, and the human should bring their laptop to show the latest LLM demo. The human sought advice on what would be a good demo to show the customer and why, in the context of their lunch meeting to discuss the latest in AI. The AI suggested an LLM text generation demo, noting it\'s impressive and relevant.', additional_kwargs={}, response_metadata={})]

---
# Summary of Your LangChain Learning Journey: Memory

In this notebook, you've embarked on a comprehensive exploration of **Memory** in LangChain, a critical component for building intelligent, stateful conversational AI. Here's a recap of the key concepts you've mastered:

**1. The "Why" of Memory:**

*   You started by confronting the inherent **statelessness** of LLMs, observing that without a memory mechanism, they cannot recall past interactions. This highlighted the fundamental need for memory in conversational applications.

**2. Building Memory from Scratch:**

*   You gained a foundational understanding of memory by implementing a **Simple Memory** using a basic Python list. This hands-on exercise demystified the core principles of storing and retrieving conversation history.

**3. A Look at Legacy Memory Modules (Deprecated):**

*   You journeyed through LangChain's history, experimenting with several of its original, now-deprecated memory modules. This provided valuable context on the evolution of memory management within the library. These legacy modules include:
    *   `ConversationBufferMemory`
    *   `ConversationBufferWindowMemory`
    *   `ConversationTokenBufferMemory`
    *   `ConversationSummaryBufferMemory`

**4. The Modern Approach: `RunnableWithMessageHistory`**

*   You then graduated to the current, recommended approach for managing chat history: `RunnableWithMessageHistory`. You learned how this powerful tool simplifies the process of making your chains stateful and managing conversation histories for multiple sessions.

**5. Advanced Customization with `BaseChatMessageHistory`**

*   Finally, you delved into advanced customization by creating your own **custom memory class**. By inheriting from `BaseChatMessageHistory`, you learned how to implement bespoke logic for storing and summarizing conversation history, giving you complete control over the memory management process.

**Key Takeaways:**

Your journey through this notebook has equipped you with the skills to:

*   **Grasp** the essential role of memory in creating coherent and context-aware conversational agents.
*   **Appreciate** the evolution of memory management in LangChain, from its legacy components to its modern, more powerful tools.
*   **Implement** both basic and advanced memory solutions, from simple lists to custom, summarization-based memory classes.
*   **Confidently apply** the `RunnableWithMessageHistory` class to build robust, scalable, and session-aware conversational applications.

You are now well-prepared to build more sophisticated, engaging, and human-like conversational experiences with LangChain. Excellent work!

* * *
# Key Commands and Imports to Remember

### Python Libraries:
- **`import os`**: Interacts with the operating system, mainly for accessing environment variables.
- **`from dotenv import load_dotenv`**: Loads environment variables from a `.env` file to securely manage API credentials.

### LangChain Core Libraries (Modern Approach):
- **`from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder`**: Used to create flexible and stateful prompt templates. `MessagesPlaceholder` is key for inserting chat history.
- **`from langchain_core.output_parsers.string import StrOutputParser`**: A simple parser to get the string content from the LLM's output.
- **`from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory`**: `InMemoryChatMessageHistory` is a simple, in-memory way to store chat history. `BaseChatMessageHistory` is the base class for creating custom chat history classes.
- **`from langchain_core.messages import BaseMessage`**: The base class for all message types, useful when creating custom history objects.
- **`from langchain_core.runnables.history import RunnableWithMessageHistory`**: The main class for wrapping a chain to make it stateful.

### Legacy LangChain Libraries (Deprecated):
- **`from langchain.chains import ConversationChain`**: The older chain for facilitating conversations with memory.
- **`from langchain.memory import ...`**: This module contains various **deprecated** memory types:
  - `ConversationBufferMemory`
  - `ConversationBufferWindowMemory`
  - `ConversationTokenBufferMemory`
  - `ConversationSummaryBufferMemory`

### Key LangChain Classes and Functions:
- **`init_chat_model(...)`**: Initializes your chosen chat model.
- **`ChatPromptTemplate.from_messages([...])`**: Creates a prompt template.
- **`RunnableWithMessageHistory(...)`**: Wraps a runnable/chain to give it stateful memory. You must provide the runnable, a `get_session_history` function, and specify the input/history keys.
- **`history_chat.invoke({"input": ...}, config={"session_id": ...})`**: Runs the stateful chain for a specific session.
- **`class CustomHistory(BaseChatMessageHistory): ...`**: Inherit from `BaseChatMessageHistory` to create your own custom memory management logic.